In [56]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, precision_recall_fscore_support

In [57]:
# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [58]:

# # Generate synthetic data
# raw_data = np.random.random((1000, 100))
# raw_data = pd.DataFrame(raw_data, columns=[f'feature_{i}' for i in range(100)])
# raw_data['target'] = np.random.randint(0, 4, size=(1000,))
# raw_data['Timestamp'] = pd.date_range(start='1/1/2020', periods=1000, freq='h')
# raw_data['Timestamp'] = raw_data['Timestamp'].astype(str)
# raw_data['Timestamp'] = pd.to_datetime(raw_data['Timestamp'])
# raw_data = raw_data.set_index('Timestamp')



# Load dataset
raw_data = pd.read_csv('dummy.csv')



NoOfCUs = 4
NoOfDUs = 4

# Creating Topology
topology = {}

# Form the graph where srscu0 connects to srsdu0, srscu1 to srsdu1, and so on
for i in range(min(NoOfCUs, NoOfDUs)):  # Prevent index errors
    topology[f"srscu{i}"] = [f"srsdu{i}"]

# Display the graph
print(topology)


common_features = raw_data.columns.tolist()
container_specific_features = {}

# Loop through and remove columns containing specific substrings
for i in range(NoOfDUs+1):
    common_features = [col for col in common_features if f"srscu{i}" not in col and f"srsdu{i}" not in col]

# Store container-specific dataframes instead of lists
for i in range(NoOfCUs+1):
    container_specific_features[f'srscu{i}'] = raw_data[[col for col in raw_data.columns.tolist() if f"srscu{i}" in col]]

for i in range(NoOfDUs+1):
    container_specific_features[f'srsdu{i}'] = raw_data[[col for col in raw_data.columns.tolist() if f"srsdu{i}" in col]]

# # Print the remaining features
# print(len(common_features), common_features)

# print("Before:")

# # Print container-specific features (as dataframes now)
# for i in range(NoOfCUs):
#     print(f"srscu{i}:")
#     print(container_specific_features[f'srscu{i}'].shape)
#     print(container_specific_features[f'srscu{i}'].head())

# for i in range(NoOfDUs):
#     print(f"srsdu{i}:")
#     print(container_specific_features[f'srsdu{i}'].shape)
#     print(container_specific_features[f'srsdu{i}'].head())

# Filter out columns containing 'stepStress' from the container-specific dataframes
for i in range(NoOfCUs):
    container_specific_features[f'srscu{i}'] = container_specific_features[f'srscu{i}'].loc[:, ~container_specific_features[f'srscu{i}'].columns.str.contains('stepStress')]

for i in range(NoOfDUs):
    container_specific_features[f'srsdu{i}'] = container_specific_features[f'srsdu{i}'].loc[:, ~container_specific_features[f'srsdu{i}'].columns.str.contains('stepStress')]

# print("After:")

# # Print container-specific features (after filtering)
# for i in range(NoOfCUs):
#     print(f"srscu{i}:")
#     print(container_specific_features[f'srscu{i}'].shape)
#     print(container_specific_features[f'srscu{i}'].head())

# for i in range(NoOfDUs):
#     print(f"srsdu{i}:")
#     print(container_specific_features[f'srsdu{i}'].shape)
#     print(container_specific_features[f'srsdu{i}'].head())


# Iterate through the topology and combine features
combined_samples = {}

for CU in topology.keys():
    # The CU container-specific features
    CU_features = container_specific_features[CU]
    
    # The connected DUs (from topology)
    connected_DUs = topology[CU]
    
    # Add CU-specific features to the combined list
    CU_features_list = CU_features.columns.tolist()
    
    # Extract the CU stress type column (if exists)
    CU_stressType = f'{CU}_stressType' if f'{CU}_stressType' in CU_features.columns else None
    
    # Add DU-specific features to the combined list for each connected DU
    for DU in connected_DUs:
        # Ensure DU exists in container_specific_features
        if DU in container_specific_features:
            DU_features = container_specific_features[DU]
            DU_features_list = DU_features.columns.tolist()

            # Combine CU and DU features (remove the stress type columns from features)
            combined_features = common_features.copy()  # Start with the common features
            
            # Modify these lines:
            combined_features.extend(CU_features_list)  # Keep all CU features
            combined_features.extend(DU_features_list)  # Keep all DU features

            

            # Extract targets and remove them from features
            targets = [col for col in combined_features if '_stressType' in col]

            # To keep stressType columns temporarily:
            combined_samples[(CU, DU)] = {
                'features': list(set(combined_features) - set(targets)),  # Include targets in features temporarily
                'targets': list(set(targets))
            }

        else:
            print(f"Error: {DU} not found in container_specific_features!")
            continue  # Skip this DU if not found in container_specific_features
    
print(combined_samples)
# Print the results for each CU-DU pair and its combined features
for (CU, DU), sample in combined_samples.items():
    print(f"Host and CU: {CU}, DU: {DU} - Combined Features:")
    print(f"Number of Features: {len(sample['features'])}")
    print(f"Number of Targets: {len(sample['targets'])}")
    print(sample['features'][:10])  # Print first 10 features as a preview
    print("----" * 10)


{'srscu0': ['srsdu0'], 'srscu1': ['srsdu1'], 'srscu2': ['srsdu2'], 'srscu3': ['srsdu3']}
{('srscu0', 'srsdu0'): {'features': ['irate(node_schedstat_timeslices_total{instance="node-exporter:9100",job="node"}[10s])', 'irate(node_network_receive_drop_total{instance="node-exporter:9100",job="node"}[10s])', 'sum  (irate(container_cpu_user_seconds_total{name="open5gs_5gc",instance="cadvisor:8080"}[5m]) * 100)', 'rate(node_pressure_io_waiting_seconds_total{instance="node-exporter:9100",job="node"}[10s])', 'sum  (irate(container_network_transmit_bytes_total{name="srsdu0",instance="cadvisor:8080",interface="eth0"}[5m]))', 'sum(irate(node_cpu_seconds_total{instance="node-exporter:9100",job="node", mode="nice"}[10s])) / scalar(count(count(node_cpu_seconds_total{instance="node-exporter:9100",job="node"}) by (cpu)))', 'node_filefd_allocated{instance="node-exporter:9100",job="node"}', 'irate(node_vmstat_pgpgout{instance="node-exporter:9100",job="node"}[10s])', 'sum(irate(container_cpu_user_seconds_t

In [59]:


# # Define target features to aggregate into a single target column
# target_features = [
#     'srsdu1_stressType', 'srsdu2_stressType', 'srsdu0_stressType', 'srsdu3_stressType',
#     'srscu0_stressType', 'srscu2_stressType', 'srscu3_stressType', 'srscu1_stressType'
# ]


# # Create a unified target column based on the most frequent non-zero stress type
# for idx, sample in raw_data.iterrows():
#     targets = [sample[target] for target in target_features]
#     non_zero_targets = [value for value in targets if value != 0]
#     raw_data.at[idx, 'target'] = max(set(non_zero_targets), key=non_zero_targets.count) if non_zero_targets else 0


for idx, sample in raw_data.iterrows():
    for itr in ['key']:  # Loop through the list of columns you want to modify
        val = str(sample[itr])
        
        # Remove the substring 'bsr' from the value
        val = val.replace('bsr', '')
        
        # Convert the modified value to an integer, if possible
        try:
            raw_data.at[idx, itr] = int(val)
        except ValueError:
            # Handle the case where the value cannot be converted to an integer
            raw_data.at[idx, itr] = 0  # Or set it to some default value

# # Drop original target feature columns
# raw_data = raw_data.drop(columns=target_features)

dropFeatures = ['srscu3_stepStress', 'srsdu3_stepStress', 'srscu2_stepStress', 'srsdu2_stepStress', 'srscu1_stepStress', 'srsdu1_stepStress', 'srscu0_stepStress', 'srsdu0_stepStress']
raw_data = raw_data.drop(columns=dropFeatures)

container_available = ['srsdu0', 'srsdu1', 'srsdu2', 'srsdu3', 'srscu0', 'srscu1', 'srscu2', 'srscu3']



# Set 0: Features where none of the exclude terms except 'srscu0' and 'srsdu0' are in the column names
exclude_terms_set0 = [term for term in container_available if term not in ['srscu0', 'srsdu0']]
set0_raw_data = raw_data.loc[:, ~raw_data.columns.str.contains('|'.join(exclude_terms_set0))]

# Set 1: Features where none of the exclude terms except 'srscu1' and 'srsdu1' are in the column names
exclude_terms_set1 = [term for term in container_available if term not in ['srscu1', 'srsdu1']]
set1_raw_data = raw_data.loc[:, ~raw_data.columns.str.contains('|'.join(exclude_terms_set1))]

# Set 2: Features where none of the exclude terms except 'srscu2' and 'srsdu2' are in the column names
exclude_terms_set2 = [term for term in container_available if term not in ['srscu2', 'srsdu2']]
set2_raw_data = raw_data.loc[:, ~raw_data.columns.str.contains('|'.join(exclude_terms_set2))]

# Set 3: Features where none of the exclude terms except 'srscu3' and 'srsdu3' are in the column names
exclude_terms_set3 = [term for term in container_available if term not in ['srscu3', 'srsdu3']]
set3_raw_data = raw_data.loc[:, ~raw_data.columns.str.contains('|'.join(exclude_terms_set3))]

print(f'Shape of rawdata is {raw_data.shape}')
print(f'Shape of set0 is {set0_raw_data.shape}')
print(f'Shape of set1 is {set1_raw_data.shape}')
print(f'Shape of set2 is {set2_raw_data.shape}')
print(f'Shape of set3 is {set3_raw_data.shape}')


Shape of rawdata is (2999, 397)
Shape of set0 is (2999, 307)
Shape of set1 is (2999, 307)
Shape of set2 is (2999, 307)
Shape of set3 is (2999, 307)


# Only working on Set0 (CU0, DU0)

In [60]:
for feature in set0_raw_data.columns:
    if "Type" in feature:
        print(feature)
col0 = [feat for feat in set0_raw_data.columns if feat not in ['srsdu0_stressType', 'srscu0_stressType']]
col1 = [feat for feat in set1_raw_data.columns if feat not in ['srsdu1_stressType', 'srscu1_stressType']]
col2 = [feat for feat in set2_raw_data.columns if feat not in ['srsdu2_stressType', 'srscu2_stressType']]
col3 = [feat for feat in set3_raw_data.columns if feat not in ['srsdu3_stressType', 'srscu3_stressType']]

print(f'Shape of set0 is {len(col0)}')
print(f'Shape of set1 is {len(col1)}')
print(f'Shape of set2 is {len(col2)}')
print(f'Shape of set3 is {len(col3)}')

container_specific_columns = list(set(col0) - set(col1))
container_specific_columns

# Extract column names as a flat list
common_features = raw_data.columns.tolist()
container_specific_features = {}

# Loop through and remove columns containing specific substrings
for i in range(5):
    # Filter out columns that contain "srscu{i}" or "srsdu{i}" from common_features
    common_features = [col for col in common_features if f"srscu{i}" not in col and f"srsdu{i}" not in col]

    # Store container-specific features (columns that DO contain "srscu{i}" or "srsdu{i}")
    container_specific_features[f'srscu{i}'] = [col for col in raw_data.columns.tolist() if f"srscu{i}" in col]
    container_specific_features[f'srsdu{i}'] = [col for col in raw_data.columns.tolist() if f"srsdu{i}" in col]

# Print the remaining features
print(len(common_features), common_features)

# Print container-specific features
print(len(container_specific_features['srscu0']), container_specific_features['srscu0'])
print(len(container_specific_features['srsdu0']), container_specific_features['srsdu0'])


srsdu0_stressType
srscu0_stressType
Shape of set0 is 305
Shape of set1 is 305
Shape of set2 is 305
Shape of set3 is 305
277 ['Timestamp', 'irate(node_pressure_cpu_waiting_seconds_total{instance="node-exporter:9100",job="node"}[10s])', 'irate(node_pressure_memory_waiting_seconds_total{instance="node-exporter:9100",job="node"}[10s])', 'irate(node_pressure_io_waiting_seconds_total{instance="node-exporter:9100",job="node"}[10s])', '100 * (1 - avg(rate(node_cpu_seconds_total{mode="idle", instance="node-exporter:9100"}[10s])))', 'scalar(node_load1{instance="node-exporter:9100",job="node"}) * 100 / count(count(node_cpu_seconds_total{instance="node-exporter:9100",job="node"}) by (cpu))', '((node_memory_MemTotal_bytes{instance="node-exporter:9100", job="node"} - node_memory_MemFree_bytes{instance="node-exporter:9100", job="node"}) / node_memory_MemTotal_bytes{instance="node-exporter:9100", job="node"}) * 100', '(1 - (node_memory_MemAvailable_bytes{instance="node-exporter:9100", job="node"} / no

In [61]:
raw_data = set0_raw_data.copy()

feature_names = raw_data.columns
timestamps = raw_data.index
raw_data = raw_data.drop(columns=['Timestamp'], errors='ignore')

# **Data Preprocessing**
# Handle missing values
raw_data = raw_data.apply(lambda x: x.fillna(0) if x.isna().all() else x)
threshold = 0.6 * len(raw_data)
for col in raw_data.columns:
    if raw_data[col].isna().sum() > threshold:
        mode_value = raw_data[col].mode().iloc[0] if not raw_data[col].mode().empty else 0
        raw_data.fillna({col: mode_value}, inplace=True)
#            dataset[col].fillna(mode_value, inplace=True)
# dataset = dataset.dropna(subset=['target'])
numeric_cols = raw_data.select_dtypes(include=[np.number]).columns
raw_data[numeric_cols] = raw_data[numeric_cols].fillna(raw_data[numeric_cols].mean())





if 'srscu0_stressType' in raw_data.columns:
    print("srscu0_stressType is present")
if 'srsdu0_stressType' in raw_data.columns:
    print("srsdu0_stressType is present")

for i in range(len(raw_data)):
    if(raw_data.at[i, 'srscu0_stressType']!=0 or raw_data.at[i, 'srsdu0_stressType']!=0):
        raw_data.at[i, 'target'] = 1
    else:
        raw_data.at[i, 'target'] = 0

original_targets = [raw_data['srscu0_stressType'], raw_data['srsdu0_stressType']]
raw_data.drop(columns=['srscu0_stressType', 'srsdu0_stressType'], inplace=True)





raw_data = raw_data[raw_data['target'].isin([0, 1, 2, 3])]

# raw_data = (raw_data - raw_data.mean()) / (raw_data.std()+1)



from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from sklearn.model_selection import StratifiedKFold



train_idx = int(0.8 * len(raw_data))
raw_data_training = raw_data[:train_idx]
raw_data_testing = raw_data[train_idx:]

# Use float32 for preprocessing stability
raw_data_training = raw_data_training.astype(np.float32)
raw_data_testing = raw_data_testing.astype(np.float32)



# Convert to PyTorch tensors
features = raw_data_training.values[:, :-1]
# Step 1: Ensure features are float32 (safe for scaling and PCA)
features = features.astype(np.float32)

labels = raw_data_training['target'].values



PCA_DIMENSION = 10
# Scale features and apply PCA
scaler = MinMaxScaler(feature_range=(-1, 1))
features_scaled = scaler.fit_transform(features)

pca = PCA(n_components=PCA_DIMENSION)
features_pca = pca.fit_transform(features_scaled)

features_pca = torch.tensor(features_pca, dtype=torch.float32).to(device)
labels = torch.tensor(labels, dtype=torch.long).to(device)



# Create dataset and dataloader
dataset = TensorDataset(features_pca, labels)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Define the Generator
class Generator(nn.Module):
    def __init__(self, latent_dim, num_features, num_classes):
        super(Generator, self).__init__()
        self.latent_dim = latent_dim
        self.num_features = num_features
        self.num_classes = num_classes
        
        # Adjust the first layer to match the input dimensions
        self.model = nn.Sequential(
            nn.Linear(latent_dim + num_classes, 512),  # Adjust this to match input dimensions
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, num_features),
            nn.Tanh()
        )
    
    def forward(self, z, labels):
        # Concatenate latent vector and labels
        inputs = torch.cat([z, labels], dim=1)
        return self.model(inputs)

# Define the Discriminator
class Discriminator(nn.Module):
    def __init__(self, num_features, num_classes):
        super(Discriminator, self).__init__()
        self.num_features = num_features
        self.num_classes = num_classes
        
        self.model = nn.Sequential(
            nn.Linear(num_features + num_classes, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 128),
            nn.LeakyReLU(0.2),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x, labels):
        # Concatenate features and labels
        inputs = torch.cat([x, labels], dim=1)
        return self.model(inputs)

# Hyperparameters
latent_dim = features_pca.shape[1] - 1
num_features = features_pca.shape[1] - 1
num_classes = 2
lr = 0.0002
num_epochs = 20

# Initialize models
generator = Generator(latent_dim, num_features, num_classes).to(device)
discriminator = Discriminator(num_features, num_classes).to(device)

# Optimizers
g_optimizer = optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
d_optimizer = optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))

# Loss function
criterion = nn.BCELoss()

# Training loop
for epoch in range(num_epochs):
    for i, (real_data, real_labels) in enumerate(dataloader):
        batch_size = real_data.size(0)
        
        # Train Discriminator
        d_optimizer.zero_grad()
        
        # Real data
        real_labels_onehot = nn.functional.one_hot(real_labels, num_classes).float().to(device)
        real_validity = discriminator(real_data, real_labels_onehot)
        d_real_loss = criterion(real_validity, torch.ones_like(real_validity).to(device))
        
        # Fake data
        z = torch.randn(batch_size, latent_dim).to(device)
        fake_labels = torch.randint(0, num_classes, (batch_size,)).to(device)
        fake_labels_onehot = nn.functional.one_hot(fake_labels, num_classes).float().to(device)
        fake_data = generator(z, fake_labels_onehot)
        fake_validity = discriminator(fake_data.detach(), fake_labels_onehot)
        d_fake_loss = criterion(fake_validity, torch.zeros_like(fake_validity).to(device))
        
        d_loss = d_real_loss + d_fake_loss
        d_loss.backward()
        d_optimizer.step()
        
        # Train Generator
        g_optimizer.zero_grad()
        
        z = torch.randn(batch_size, latent_dim).to(device)
        fake_labels = torch.randint(0, num_classes, (batch_size,)).to(device)
        fake_labels_onehot = nn.functional.one_hot(fake_labels, num_classes).float().to(device)
        fake_data = generator(z, fake_labels_onehot)
        fake_validity = discriminator(fake_data, fake_labels_onehot)
        g_loss = criterion(fake_validity, torch.ones_like(fake_validity).to(device))
        
        g_loss.backward()
        g_optimizer.step()
        
        if i % 100 == 0:
            print(f"Epoch [{epoch}/{num_epochs}] Batch [{i}/{len(dataloader)}] "
                  f"D Loss: {d_loss.item():.4f}, G Loss: {g_loss.item():.4f}")


# Testing
# Convert to PyTorch tensors
test_z = torch.FloatTensor(raw_data_testing.values[:, :-1]).to(device)
test_labels = torch.LongTensor(raw_data_testing['target'].values).to(device)
test_labels_onehot = nn.functional.one_hot(test_labels, num_classes).float().to(device)
test_data = generator(test_z, test_labels_onehot)

torch.save(generator.state_dict(), 'generator.pth')
torch.save(discriminator.state_dict(), 'discriminator.pth')

# Evaluate the generated data for anomaly detection
def evaluate_anomaly_detection(generator, real_data, labels, num_samples=1000):
    # Generate synthetic data
    z = torch.randn(num_samples, latent_dim).to(device)
    synthetic_labels = torch.randint(0, num_classes, (num_samples,)).to(device)
    synthetic_labels_onehot = nn.functional.one_hot(synthetic_labels, num_classes).float().to(device)
    synthetic_data = generator(z, synthetic_labels_onehot)
    
    # Combine real and synthetic data
    all_data = torch.cat([real_data, synthetic_data], dim=0)
    all_labels = torch.cat([labels, synthetic_labels], dim=0)
    
    # Use discriminator to classify real vs synthetic
    with torch.no_grad():
        predictions = discriminator(all_data, nn.functional.one_hot(all_labels, num_classes).float().to(device))
    
    # Convert predictions to binary (0 for synthetic, 1 for real)
    predictions = (predictions > 0.5).float()
    
    # Calculate anomaly detection metrics
    real_labels = torch.ones(real_data.size(0)).to(device)
    synthetic_labels = torch.zeros(synthetic_data.size(0)).to(device)
    true_labels = torch.cat([real_labels, synthetic_labels], dim=0)
    
    auc_roc = roc_auc_score(true_labels.cpu().numpy(), predictions.cpu().numpy())
    precision, recall, f1, _ = precision_recall_fscore_support(true_labels.cpu().numpy(), predictions.cpu().numpy(), average='binary')
    
    return auc_roc, precision, recall, f1


# Evaluate anomaly detection performance
auc_roc, precision, recall, f1 = evaluate_anomaly_detection(generator, features, labels)


print(f"AUC-ROC: {auc_roc:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


srscu0_stressType is present
srsdu0_stressType is present


/tmp/ipykernel_5309/2271811470.py:31: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  raw_data.at[i, 'target'] = 1


RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x12 and 11x256)